## Step 1: Setup (Run Once)

In [1]:
# Install packages
!pip install pandas numpy matplotlib seaborn scikit-learn --quiet

# Import everything
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from calendar import month_name, monthrange
import warnings
import random

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# Categories
categories = ['Food', 'Transportation', 'Entertainment', 'Shopping', 
              'Bills', 'Healthcare', 'Education', 'Other']

# Storage
all_expenses = pd.DataFrame(columns=['Year', 'Month', 'Date', 'Category', 'Amount', 'Description'])
monthly_salary = 0
budgets = {cat: 0 for cat in categories}
salary_history = {}  # Track salary changes

# Financial Goals Storage
financial_goals = []

# Seasonal tips
seasonal_tips = {
    'Winter': ['heating bills increase', 'winter clothes', 'holiday expenses'],
    'Spring': ['spring cleaning', 'outdoor activities', 'lighter clothing'],
    'Summer': ['cooling costs', 'vacation expenses', 'summer activities'],
    'Autumn': ['back-to-school costs', 'heater preparation', 'autumn clothing']
}

# Seasonal multipliers for realistic patterns
seasonal_multipliers = {
    'Winter': {
        'Bills': 1.35,        # +35% heating
        'Healthcare': 1.25,   # +25% flu season
        'Food': 1.10,         # +10% 
        'Shopping': 1.15,     # +15% winter clothes
        'Entertainment': 0.85 # -15% less outdoor
    },
    'Spring': {
        'Entertainment': 1.25, # +25% outdoor activities
        'Transportation': 1.10, # +10% more travel
        'Shopping': 1.10,      # +10% spring clothes
        'Bills': 0.85          # -15% no heating/cooling
    },
    'Summer': {
        'Entertainment': 1.30, # +30% vacation
        'Bills': 1.20,         # +20% cooling/AC
        'Transportation': 1.15, # +15% vacation travel
        'Education': 0.80      # -20% summer break
    },
    'Autumn': {
        'Education': 1.40,     # +40% new semester
        'Shopping': 1.20,      # +20% back-to-school
        'Bills': 0.90          # -10% mild weather
    }
}

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

def get_seasonal_multiplier(category, season):
    """Get seasonal multiplier for a category, default to 1.0"""
    return seasonal_multipliers.get(season, {}).get(category, 1.0)

def calculate_historical_salary(current_salary, year, month):
    """
    Calculate what salary was in the past, considering 15% raise every August.
    Working backwards from Nov 2025.
    """
    # Count how many Augusts have passed since the target month
    # Current: Nov 2025
    # Aug 2025: 1 raise ago (divide by 1.15)
    # Aug 2024: 2 raises ago (divide by 1.15^2)
    # Aug 2023: 3 raises ago (would be before our data starts)
    
    target_date = year * 12 + month
    current_date = 2025 * 12 + 11  # Nov 2025
    
    # Count Augusts between target and current
    august_count = 0
    
    # After Aug 2025
    if target_date >= 2025 * 12 + 8:  # Aug 2025 or later
        august_count = 0
    # After Aug 2024, before Aug 2025
    elif target_date >= 2024 * 12 + 8:  # Aug 2024 to Jul 2025
        august_count = 1
    # Before Aug 2024
    else:  # Before Aug 2024
        august_count = 2
    
    # Calculate historical salary
    historical_salary = current_salary / (1.15 ** august_count)
    
    return historical_salary

def calculate_inflation_multiplier(months_ago):
    """
    Calculate compound inflation multiplier.
    1% per month, so expenses grow over time.
    months_ago: how many months before Nov 2025
    """
    # Inflation factor: (1.01)^months means expenses were lower in the past
    # So we divide: historical_expense = current / (1.01^months_ago)
    return 1.0 / (1.01 ** months_ago)

print("✓ Setup Complete!")
print("\n📊 Categories: " + ", ".join(categories))
print("\n💰 Optimized for Kurdistan living expenses (~$500-900/month)")
print("📅 Will auto-generate 24 months of history (Nov 2023 - Oct 2025)!")
print("🌦️  WITH REALISTIC SEASONAL PATTERNS!")
print("\n📈 Economic Model:")
print("   - 15% salary increase every August")
print("   - 1% monthly inflation on expenses")
print("\n🎯 NEW FEATURES:")
print("   - CSV import for expenses")
print("   - Financial goal tracking")


✓ Setup Complete!

📊 Categories: Food, Transportation, Entertainment, Shopping, Bills, Healthcare, Education, Other

💰 Optimized for Kurdistan living expenses (~$500-900/month)
📅 Will auto-generate 24 months of history (Nov 2023 - Oct 2025)!
🌦️  WITH REALISTIC SEASONAL PATTERNS!

📈 Economic Model:
   - 15% salary increase every August
   - 1% monthly inflation on expenses

🎯 NEW FEATURES:
   - CSV import for expenses
   - Financial goal tracking


## Step 2: Enter Salary & Set November Budget

In [ ]:
print("\n" + "="*60)
print("November 2025 - SALARY & BUDGET SETUP")
print("="*60)

current_month = 11  # November
current_year = 2025
season = get_season(current_month)

print(f"\n Current Month: November 2025")
print(f" Season: {season}")
print(f"\n Seasonal Tips:")
for tip in seasonal_tips[season]:
    print(f"   • {tip}")

# Get salary
print("\n" + "-"*60)
salary_input = input("\nEnter your CURRENT monthly salary (Nov 2025): $")

try:
    monthly_salary = float(salary_input)
    
    print(f"\n Current Salary (Nov 2025): ${monthly_salary:,.2f}")
    print("\n Calculating salary history with 15% raises every August...")
    
    # Show salary progression
    salary_aug_2025 = monthly_salary  # No raise yet (last raise was Aug 2025)
    salary_aug_2024 = monthly_salary / 1.15
    salary_before_aug_2024 = salary_aug_2024 / 1.15
    
    print(f"   Nov 2023 - Jul 2024: ${salary_before_aug_2024:,.2f}")
    print(f"   Aug 2024 - Jul 2025: ${salary_aug_2024:,.2f} (15% raise!)")
    print(f"   Aug 2025 - Now:      ${salary_aug_2025:,.2f} (15% raise!)")
    
    print("\n Generating 2 YEARS of history (Nov 2023 - Oct 2025)...")
    print(" With salary growth + monthly inflation + seasonal patterns...")
    
    # Generate realistic Kurdistan expenses for past 24 months
    descriptions = {
        'Food': ['Grocery', 'Restaurant', 'Fast food', 'Bakery', 'Market'],
        'Transportation': ['Taxi', 'Gas', 'Bus', 'Maintenance', 'Parking'],
        'Entertainment': ['Movie', 'Cafe', 'Gaming', 'Internet', 'Sports'],
        'Shopping': ['Clothes', 'Phone', 'Shoes', 'Home items', 'Accessories'],
        'Bills': ['Electricity', 'Internet', 'Phone', 'Water', 'Rent'],
        'Healthcare': ['Medicine', 'Doctor', 'Dental', 'Pharmacy', 'Vitamins'],
        'Education': ['Books', 'Course', 'Supplies', 'Tuition', 'Materials'],
        'Other': ['Gift', 'Donation', 'Repair', 'Misc', 'Emergency']
    }
    
    # Category percentages (your fixed percentages)
    category_pcts = {
        'Food': 0.30,          # 30%
        'Transportation': 0.15, # 15%
        'Entertainment': 0.10,  # 10%
        'Shopping': 0.10,       # 10%
        'Bills': 0.15,          # 15%
        'Healthcare': 0.05,     # 5%
        'Education': 0.02,      # 2%
        'Other': 0.02           # 2%
    }
    
    # Generate 24 months (Nov 2023 to Oct 2025)
    start_year = 2023
    start_month = 11  # November
    
    for month_offset in range(24):
        # Calculate year and month
        total_months = start_month + month_offset
        year = start_year + (total_months - 1) // 12
        month = ((total_months - 1) % 12) + 1
        
        # Calculate how many months ago this was from Nov 2025
        months_from_now = 23 - month_offset
        
        # Get historical salary for this month
        hist_salary = calculate_historical_salary(monthly_salary, year, month)
        salary_history[f"{year}-{month:02d}"] = hist_salary
        
        # Get inflation multiplier (expenses were lower in the past)
        inflation_mult = calculate_inflation_multiplier(months_from_now)
        
        # Get season for this month
        month_season = get_season(month)
        
        # Base monthly spending (90% of salary with ±5% variation)
        base_spending = hist_salary * 0.90 * random.uniform(0.95, 1.05)
        
        # Apply inflation multiplier
        base_spending *= inflation_mult
        
        # Number of expenses for this month (50-70)
        num_expenses = random.randint(50, 70)
        
        month_total = 0
        
        for _ in range(num_expenses):
            # Pick random category
            cat = random.choice(categories)
            
            # Base amount for this category
            base_cat_amount = base_spending * category_pcts[cat] / (num_expenses / len(categories))
            
            # Apply seasonal multiplier
            seasonal_mult = get_seasonal_multiplier(cat, month_season)
            
            # Final amount with variation
            amount = base_cat_amount * seasonal_mult * random.uniform(0.5, 1.8)
            
            # Keep amounts reasonable for Kurdistan
            if amount < 2:
                amount = random.uniform(2, 8)
            elif amount > 100:
                amount = random.uniform(20, 80)
            
            # Random day in month
            max_day = monthrange(year, month)[1]
            day = random.randint(1, max_day)
            
            # Add expense
            new_row = pd.DataFrame([{
                'Year': year,
                'Month': month,
                'Date': f"{year}-{month:02d}-{day:02d}",
                'Category': cat,
                'Amount': round(amount, 2),
                'Description': random.choice(descriptions[cat])
            }])
            all_expenses = pd.concat([all_expenses, new_row], ignore_index=True)
            month_total += amount
        
        # Print progress every 6 months
        if (month_offset + 1) % 6 == 0 or month_offset == 23:
            print(f"    {month_name[month]} {year}: {num_expenses} expenses, ${month_total:.2f} (salary: ${hist_salary:.2f})")
    
    print("\n History generated with SEASONAL PATTERNS + ECONOMIC GROWTH!")
    
    # Budget setup
    print("\n" + "-"*60)
    print("How to set November 2025 budget?")
    print("-"*60)
    print("1 = Auto-divide (recommended for Kurdistan)")
    print("2 = Set manually")
    print()
    
    choice = input("Enter choice (1 or 2): ")
    
    if choice == '1':
        print("\n Setting realistic Kurdistan budget...")
        
        # Budget = 80% of salary
        total_budget = monthly_salary * 0.80
        
        # Set budgets using your percentages
        budgets['Food'] = total_budget * 0.30
        budgets['Transportation'] = total_budget * 0.15
        budgets['Entertainment'] = total_budget * 0.10
        budgets['Shopping'] = total_budget * 0.10
        budgets['Bills'] = total_budget * 0.15
        budgets['Healthcare'] = total_budget * 0.05
        budgets['Education'] = total_budget * 0.02
        budgets['Other'] = total_budget * 0.02
        
        print("\nNovember 2025 Budget:")
        for cat in categories:
            pct = (budgets[cat] / total_budget) * 100
            print(f"   {cat:<14} : ${budgets[cat]:>7.2f} ({pct:>4.1f}%)")
        
        savings = monthly_salary - total_budget
        savings_pct = (savings / monthly_salary) * 100
        
        print("\n" + "="*60)
        print(f"Total Budget: ${total_budget:.2f} (80% of salary)")
        print(f"For Savings:  ${savings:.2f} (20%)")
        print("="*60)
    
    else:
        print("\n Setting budgets manually...")
        total_budget = 0
        
        for cat in categories:
            while True:
                try:
                    amt = float(input(f"{cat}: $"))
                    budgets[cat] = amt
                    total_budget += amt
                    break
                except:
                    print("Please enter a valid number")
        
        print(f"\n Total Budget: ${total_budget:.2f}")
        print(f" Savings: ${monthly_salary - total_budget:.2f}")

except ValueError:
    print("Invalid input! Please enter numbers only.")


November 2025 - SALARY & BUDGET SETUP

 Current Month: November 2025
 Season: Autumn

 Seasonal Tips:
   • back-to-school costs
   • heater preparation
   • autumn clothing

------------------------------------------------------------



Enter your CURRENT monthly salary (Nov 2025): $ 1000



 Current Salary (Nov 2025): $1,000.00

 Calculating salary history with 15% raises every August...
   Nov 2023 - Jul 2024: $756.14
   Aug 2024 - Jul 2025: $869.57 (15% raise!)
   Aug 2025 - Now:      $1,000.00 (15% raise!)

 Generating 2 YEARS of history (Nov 2023 - Oct 2025)...
 With salary growth + monthly inflation + seasonal patterns...
    April 2024: 64 expenses, $690.22 (salary: $756.14)
    October 2024: 69 expenses, $695.40 (salary: $869.57)
    April 2025: 62 expenses, $796.03 (salary: $869.57)
    October 2025: 61 expenses, $1028.63 (salary: $1000.00)

 History generated with SEASONAL PATTERNS + ECONOMIC GROWTH!

------------------------------------------------------------
How to set November 2025 budget?
------------------------------------------------------------
1 = Auto-divide (recommended for Kurdistan)
2 = Set manually



Enter choice (1 or 2):  1



 Setting realistic Kurdistan budget...

November 2025 Budget:
   Food           : $ 240.00 (30.0%)
   Transportation : $ 120.00 (15.0%)
   Entertainment  : $  80.00 (10.0%)
   Shopping       : $  80.00 (10.0%)
   Bills          : $ 120.00 (15.0%)
   Healthcare     : $  40.00 ( 5.0%)
   Education      : $  16.00 ( 2.0%)
   Other          : $  16.00 ( 2.0%)

Total Budget: $800.00 (80% of salary)
For Savings:  $200.00 (20%)


## Step 3: Add November 2025 Expenses

In [ ]:
print("\n" + "="*60)
print(f"ADD EXPENSES - November 2025")
print("="*60)

print("\nHow would you like to add expenses?")
print("   1. Generate expenses automatically")
print("   2. Add expenses manually")
print("   3. Import expenses from CSV file")

mode = input("\nSelect option (1, 2, or 3): ").strip()

if mode in ['1', '2'] and monthly_salary == 0:
    print("\n⚠️ Please run Step 2 first to set salary and budget!")
    print("   (Or use option 3 to import CSV without setting salary)")
elif mode in ['1', '2']:
    # Calculate the correct budget (80% of salary for expenses, 20% for savings)
    savings_amount = monthly_salary * 0.20
    total_budget = monthly_salary - savings_amount  # This should be $800 for $1000 salary
    
    if mode == '1':
        # AUTOMATIC EXPENSE GENERATION
        print("\n" + "-"*60)
        print("GENERATING AUTOMATIC EXPENSES...")
        print("-"*60)
        
        import random
        
        # Define realistic expense patterns for each category
        # IMPORTANT: Categories must match: Food, Transportation, Entertainment, Shopping, Bills, Healthcare, Education, Other
        expense_patterns = {
            'Food': [(6, 'Groceries', 0.65), (4, 'Restaurants', 0.35)],
            'Transportation': [(3, 'Gas', 0.70), (2, 'Public Transit', 0.30)],
            'Entertainment': [(2, 'Streaming Services', 0.25), (3, 'Movies/Events', 0.50), (2, 'Hobbies', 0.25)],
            'Shopping': [(3, 'Clothing', 0.50), (2, 'Electronics', 0.30), (2, 'Household Items', 0.20)],
            'Bills': [(1, 'Rent', 0.70), (1, 'Utilities', 0.20), (1, 'Internet', 0.10)],
            'Healthcare': [(1, 'Insurance', 0.70), (2, 'Pharmacy', 0.30)],
            'Education': [(1, 'Tuition/Books', 0.80), (1, 'Supplies', 0.20)],
            'Other': [(2, 'Gifts', 0.50), (2, 'Miscellaneous', 0.50)]
        }
        
        expense_count = 0
        
        for category in categories:
            if category in budgets and budgets[category] > 0:
                # Get the percentage this category represents
                category_budget_from_dict = budgets[category]
                
                # Scale it up to match the actual $800 budget
                # If budgets sum to $712, we need to multiply by (800/712)
                scaling_factor = total_budget / sum(budgets.values())
                actual_category_budget = category_budget_from_dict * scaling_factor
                
                patterns = expense_patterns.get(category, [(2, category, 1.0)])
                
                for num_expenses, desc_base, ratio in patterns:
                    # Distribute budget based on ratio
                    category_budget = actual_category_budget * ratio
                    
                    # Generate expenses for this pattern
                    for i in range(num_expenses):
                        # Vary the amount slightly (85-100% of average to stay within budget)
                        avg_amount = category_budget / num_expenses
                        amount = round(random.uniform(avg_amount * 0.85, avg_amount * 1.0), 2)
                        
                        # Generate random day in November
                        day = random.randint(1, 28)
                        
                        # Add expense
                        new_row = pd.DataFrame([{
                            'Year': current_year,
                            'Month': current_month,
                            'Date': f"{current_year}-{current_month:02d}-{day:02d}",
                            'Category': category,
                            'Amount': amount,
                            'Description': desc_base if num_expenses == 1 else f"{desc_base} #{i+1}"
                        }])
                        all_expenses = pd.concat([all_expenses, new_row], ignore_index=True)
                        expense_count += 1
        
        # Show summary with correct budget
        nov_expenses = all_expenses[(all_expenses['Year'] == current_year) & 
                                   (all_expenses['Month'] == current_month)]
        nov_total = nov_expenses['Amount'].sum()
        
        print(f"\n" + "="*60)
        print(f"✓ Generated {expense_count} automatic expenses!")
        print(f"✓ Monthly Salary: ${monthly_salary:.2f}")
        print(f"✓ Planned Savings (20%): ${savings_amount:.2f}")
        print(f"✓ Budget for Expenses (80%): ${total_budget:.2f}")
        print(f"✓ November Expenses: ${nov_total:.2f}")
        print(f"✓ Budget usage: {(nov_total/total_budget*100):.1f}%")
        print(f"✓ Remaining: ${(total_budget - nov_total):.2f}")
        print("="*60)
        
    elif mode == '2':
        # MANUAL EXPENSE ENTRY
        print("\n📝 Categories:")
        for i, cat in enumerate(categories, 1):
            print(f"   {i}. {cat}")
        
        print("\nEnter expenses one by one (type 'done' when finished)")
        print("-"*60)
        
        expense_count = 0
        
        while True:
            cat_input = input("\nCategory (1-8 or name, or 'done'): ").strip()
            
            if cat_input.lower() == 'done':
                break
            
            # Get category
            if cat_input.isdigit() and 1 <= int(cat_input) <= 8:
                category = categories[int(cat_input) - 1]
            elif cat_input in categories:
                category = cat_input
            else:
                print("❌ Invalid category! Try again.")
                continue
            
            # Get amount
            try:
                amount = float(input("Amount: $"))
                if amount <= 0:
                    print("❌ Amount must be positive!")
                    continue
                    
                description = input("Description (optional, press Enter to skip): ").strip()
                
                if not description:
                    description = category
                
                # Add expense for November 2025
                new_row = pd.DataFrame([{
                    'Year': current_year,
                    'Month': current_month,
                    'Date': f"{current_year}-{current_month:02d}-{datetime.now().day:02d}",
                    'Category': category,
                    'Amount': amount,
                    'Description': description
                }])
                all_expenses = pd.concat([all_expenses, new_row], ignore_index=True)
                
                expense_count += 1
                
                # Show running total for November with correct budget
                nov_expenses = all_expenses[(all_expenses['Year'] == current_year) & 
                                           (all_expenses['Month'] == current_month)]
                nov_total = nov_expenses['Amount'].sum()
                
                print(f"\n✓ Added ${amount:.2f} to {category}!")
                print(f"  November total so far: ${nov_total:.2f} / ${total_budget:.2f}")
                print(f"  Remaining budget: ${(total_budget - nov_total):.2f}")
                
            except ValueError:
                print("❌ Invalid amount! Please enter a number.")
        
        if expense_count > 0:
            print(f"\n" + "="*60)
            print(f"✓ Added {expense_count} expenses to November 2025!")
            nov_expenses = all_expenses[(all_expenses['Year'] == current_year) & 
                                       (all_expenses['Month'] == current_month)]
            nov_total = nov_expenses['Amount'].sum()
            print(f"✓ Total expenses: ${nov_total:.2f} / ${total_budget:.2f}")
            print(f"✓ Budget usage: {(nov_total/total_budget*100):.1f}%")
            print("="*60)
        else:
            print("\nℹ️  No expenses added.")

elif mode == '3':
    # CSV IMPORT - Works independently without salary/budget
    print("\n" + "-"*60)
    print("📥 CSV IMPORT INSTRUCTIONS")
    print("-"*60)
    print("\nYour CSV file must have these columns (in any order):")
    print("  • Date (format: YYYY-MM-DD, e.g., 2025-11-15)")
    print("  • Category (must match one of: " + ", ".join(categories) + ")")
    print("  • Amount (number, e.g., 45.50)")
    print("  • Description (optional, text)")
    print("\nExample CSV content:")
    print("Date,Category,Amount,Description")
    print("2025-11-01,Food,32.50,Grocery shopping")
    print("2025-11-05,Transportation,15.00,Taxi to work")
    print("2025-11-10,Bills,120.00,Electricity bill")
    print("-"*60)
    
    csv_path = input("\nEnter the full path to your CSV file: ").strip()
    
    # Remove quotes if user wrapped the path
    csv_path = csv_path.strip('"').strip("'")
    
    try:
        # Try to read the CSV
        imported_df = pd.read_csv(csv_path)
        
        # Validate columns
        required_cols = ['Date', 'Category', 'Amount']
        missing_cols = [col for col in required_cols if col not in imported_df.columns]
        
        if missing_cols:
            print(f"\n❌ Error: Missing required columns: {', '.join(missing_cols)}")
            print(f"   Found columns: {', '.join(imported_df.columns)}")
        else:
            # Add Description column if missing
            if 'Description' not in imported_df.columns:
                imported_df['Description'] = imported_df['Category']
            
            # Validate and process each row
            valid_rows = []
            errors = []
            
            for idx, row in imported_df.iterrows():
                row_num = idx + 2  # +2 because of header and 0-indexing
                
                # Validate date
                try:
                    date_obj = pd.to_datetime(row['Date'])
                    year = date_obj.year
                    month = date_obj.month
                except:
                    errors.append(f"Row {row_num}: Invalid date format '{row['Date']}'")
                    continue
                
                # Validate category
                if row['Category'] not in categories:
                    errors.append(f"Row {row_num}: Invalid category '{row['Category']}'")
                    continue
                
                # Validate amount
                try:
                    amount = float(row['Amount'])
                    if amount <= 0:
                        errors.append(f"Row {row_num}: Amount must be positive")
                        continue
                except:
                    errors.append(f"Row {row_num}: Invalid amount '{row['Amount']}'")
                    continue
                
                # Add valid row
                valid_rows.append({
                    'Year': year,
                    'Month': month,
                    'Date': row['Date'],
                    'Category': row['Category'],
                    'Amount': round(amount, 2),
                    'Description': str(row['Description'])
                })
            
            # Show results
            print(f"\n" + "="*60)
            print(f"📊 CSV IMPORT RESULTS")
            print("="*60)
            print(f"✓ Total rows in CSV: {len(imported_df)}")
            print(f"✓ Valid rows: {len(valid_rows)}")
            
            if errors:
                print(f"⚠️  Errors: {len(errors)}")
                print("\nError details:")
                for error in errors[:10]:  # Show first 10 errors
                    print(f"  • {error}")
                if len(errors) > 10:
                    print(f"  ... and {len(errors) - 10} more errors")
            
            if valid_rows:
                # Add to expenses
                new_expenses = pd.DataFrame(valid_rows)
                all_expenses = pd.concat([all_expenses, new_expenses], ignore_index=True)
                
                # Show summary by month
                print(f"\n✓ Imported {len(valid_rows)} expenses!")
                print("\nBreakdown by month:")
                monthly_import = new_expenses.groupby(['Year', 'Month']).agg({
                    'Amount': ['sum', 'count']
                }).reset_index()
                
                for _, row in monthly_import.iterrows():
                    month_name_str = month_name[int(row['Month'])]
                    year_str = int(row['Year'])
                    total = row['Amount']['sum']
                    count = row['Amount']['count']
                    print(f"  {month_name_str} {year_str}: {count} expenses, ${total:.2f}")
                
                print("\n✓ Data successfully imported!")
            else:
                print("\n❌ No valid rows to import.")
            
            print("="*60)
            
    except FileNotFoundError:
        print(f"\n❌ Error: File not found '{csv_path}'")
        print("   Please check the path and try again.")
    except Exception as e:
        print(f"\n❌ Error reading CSV: {str(e)}")

else:
    print("\n❌ Invalid option! Please select 1, 2, or 3.")



Select option (1 or 2):  1



------------------------------------------------------------
GENERATING AUTOMATIC EXPENSES...
------------------------------------------------------------

✓ Generated 41 automatic expenses!
✓ Monthly Salary: $1000.00
✓ Planned Savings (20%): $200.00
✓ Budget for Expenses (80%): $800.00
✓ November Expenses: $746.98
✓ Budget usage: 93.4%
✓ Remaining: $53.02


## Step 4: Analysis - 2 Years of Seasonal Insights

In [ ]:
if all_expenses.empty:
    print("No data yet! Run Step 2 or Step 3 (CSV import) first.")
else:
    print("\n" + "="*80)
    print(" 2-YEAR SPENDING ANALYSIS WITH SEASONAL PATTERNS")
    print("="*80)
    
    # Overall summary
    total_spent = all_expenses['Amount'].sum()
    total_expenses = len(all_expenses)
    avg_expense = all_expenses['Amount'].mean()
    
    print(f"\n Total Tracked: ${total_spent:,.2f} over 24 months")
    print(f" Total Expenses: {total_expenses:,} transactions")
    print(f" Average per expense: ${avg_expense:.2f}")
    
    # Monthly breakdown with salary info
    print("\n" + "-"*80)
    print("MONTHLY SPENDING SUMMARY (with salary):")
    print("-"*80)
    
    all_expenses['YearMonth'] = all_expenses['Year'].astype(str) + '-' + all_expenses['Month'].astype(str).str.zfill(2)
    monthly_totals = all_expenses.groupby(['Year', 'Month'])['Amount'].sum().reset_index()
    
    print(f"{'Month':<15} {'Salary':<12} {'Spending':<12} {'Season':<10}")
    print("-"*80)
    
    for _, row in monthly_totals.iterrows():
        month_name_str = month_name[int(row['Month'])]
        year_str = str(int(row['Year']))
        season = get_season(int(row['Month']))
        
        # Get salary for this month
        month_key = f"{int(row['Year'])}-{int(row['Month']):02d}"
        salary = salary_history.get(month_key, 0)
        
        print(f"{month_name_str} {year_str:<8} ${salary:>9,.2f}  ${row['Amount']:>9,.2f}  {season}")
    
    # Seasonal Analysis
    print("\n" + "="*80)
    print(" SEASONAL ANALYSIS (Average per season)")
    print("="*80)
    
    all_expenses['Season'] = all_expenses['Month'].apply(get_season)
    seasonal_spending = all_expenses.groupby('Season')['Amount'].sum() / all_expenses.groupby('Season')['Month'].nunique()
    
    seasons_order = ['Winter', 'Spring', 'Summer', 'Autumn']
    
    print("\n Average Monthly Spending by Season:")
    print("-"*80)
    for season in seasons_order:
        if season in seasonal_spending.index:
            avg = seasonal_spending[season]
            print(f" {season:<10} ${avg:>9,.2f}")
    
    # Find highest and lowest
    if len(seasonal_spending) > 0:
        highest_season = seasonal_spending.idxmax()
        lowest_season = seasonal_spending.idxmin()
        diff = seasonal_spending[highest_season] - seasonal_spending[lowest_season]
        diff_pct = (diff / seasonal_spending[lowest_season]) * 100
        
        print("\n Insights:")
        print(f"   Highest: {highest_season} (${seasonal_spending[highest_season]:,.2f}/month)")
        print(f"   Lowest:  {lowest_season} (${seasonal_spending[lowest_season]:,.2f}/month)")
        print(f"   Your {highest_season} bills average ${diff:,.2f} ({diff_pct:.1f}%) more than {lowest_season}")
    
    # Category breakdown by season
    print("\n" + "="*80)
    print(" CATEGORY SPENDING BY SEASON")
    print("="*80)
    
    category_season = all_expenses.groupby(['Season', 'Category'])['Amount'].sum().unstack(fill_value=0)
    
    # Show which categories spike in each season
    for season in seasons_order:
        if season in category_season.index:
            print(f"\n {season}:")
            season_cats = category_season.loc[season].sort_values(ascending=False)
            for cat, amt in season_cats.head(3).items():
                print(f"   {cat:<15} ${amt:>9,.2f}")
    
    # Category totals
    print("\n" + "="*80)
    print(" CATEGORY TOTALS (24 months)")
    print("="*80)
    
    cat_totals = all_expenses.groupby('Category')['Amount'].sum().sort_values(ascending=False)
    
    for cat, total in cat_totals.items():
        pct = (total / total_spent) * 100
        print(f" {cat:<15} ${total:>9,.2f} ({pct:>5.1f}%)")
    
    # November 2025 status
    nov_data = all_expenses[(all_expenses['Year'] == current_year) & 
                            (all_expenses['Month'] == current_month)]
    
    if not nov_data.empty:
        print("\n" + "="*80)
        print(" NOVEMBER 2025 STATUS (Current Month)")
        print("="*80)
        
        nov_by_cat = nov_data.groupby('Category')['Amount'].sum()
        
        print(f"\n{'Category':<15} {'Spent':<12} {'Budget':<12} {'Status':<20}")
        print("-"*80)
        
        for cat in categories:
            spent = nov_by_cat.get(cat, 0)
            budget = budgets.get(cat, 0)
            
            if budget > 0:
                pct = (spent / budget) * 100
                remaining = budget - spent
                
                if pct > 100:
                    status = f"Over by ${spent - budget:.2f}"
                elif pct > 80:
                    status = f"Warning: {pct:.0f}%"
                else:
                    status = f"OK: {pct:.0f}%"
                
                print(f"{cat:<15} ${spent:>9,.2f}  ${budget:>9,.2f}  {status}")
            else:
                print(f"{cat:<15} ${spent:>9,.2f}  No budget")
        
        nov_total_spent = nov_by_cat.sum()
        total_budget = sum(budgets.values())
        
        print("-"*80)
        print(f"{'TOTAL':<15} ${nov_total_spent:>9,.2f}  ${total_budget:>9,.2f}")
        
        if total_budget > 0:
            pct_used = (nov_total_spent / total_budget) * 100
            print(f"\n Budget Used: {pct_used:.1f}%")
            print(f" Remaining: ${total_budget - nov_total_spent:.2f}")

## Step 5: AI Feature 1 - Random Forest Prediction

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

if len(all_expenses) < 50:
    print("Need more data for AI prediction!")
else:
    print("\n" + "="*80)
    print(" AI FEATURE 1: RANDOM FOREST SPENDING PREDICTION")
    print("="*80)
    
    # Prepare data - AGGREGATE TO MONTHLY TOTALS
    df = all_expenses.copy()
    
    # Group by Year, Month, Category and sum amounts
    monthly_data = df.groupby(['Year', 'Month', 'Category'])['Amount'].sum().reset_index()
    
    # Add season
    monthly_data['Season'] = monthly_data['Month'].apply(get_season)
    
    # Encode categories
    le = LabelEncoder()
    monthly_data['Category_Encoded'] = le.fit_transform(monthly_data['Category'])
    
    # Encode seasons
    monthly_data['Season_Encoded'] = monthly_data['Season'].map({
        'Winter': 0, 'Spring': 1, 'Summer': 2, 'Autumn': 3
    })
    
    # Features (X) and target (y)
    X = monthly_data[['Year', 'Month', 'Category_Encoded', 'Season_Encoded']]
    y = monthly_data['Amount']
    
    # Train model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    # Feature importance
    importance = pd.DataFrame({
        'Feature': ['Year', 'Month', 'Category', 'Season'],
        'Importance': rf.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n Model trained on MONTHLY TOTALS!")
    print(f" Training data: {len(monthly_data)} month-category combinations")
    print("\n Feature Importance:")
    for _, row in importance.iterrows():
        print(f"   {row['Feature']:<12} {row['Importance']:.3f}")
    
    # Predict December 2025
    print("\n Predicting December 2025 MONTHLY TOTALS by category:")
    print("-"*80)
    
    dec_predictions = {}
    
    for cat in categories:
        cat_encoded = le.transform([cat])[0]
        season_encoded = 0  # Winter for December
        
        # Predict
        X_pred = [[2025, 12, cat_encoded, season_encoded]]
        pred = rf.predict(X_pred)[0]
        
        dec_predictions[cat] = pred
        print(f"   {cat:<15} ${pred:>9,.2f}")
    
    total_pred = sum(dec_predictions.values())
    print("-"*80)
    print(f"   {'TOTAL PREDICTED':<15} ${total_pred:>9,.2f}")
    print("\n This prediction uses Random Forest AI trained on 2 years of monthly totals!")

## Step 6: AI Feature 2 - Anomaly Detection

In [ ]:
from sklearn.ensemble import IsolationForest

if len(all_expenses) < 50:
    print("Need more data for anomaly detection!")
else:
    print("\n" + "="*80)
    print(" AI FEATURE 2: ANOMALY DETECTION (Unusual Expenses)")
    print("="*80)
    
    # Prepare data
    df = all_expenses.copy()
    
    # Encode categories
    le = LabelEncoder()
    df['Category_Encoded'] = le.fit_transform(df['Category'])
    
    # Features for anomaly detection
    X = df[['Amount', 'Category_Encoded', 'Month']]
    
    # Train Isolation Forest
    iso_forest = IsolationForest(contamination=0.05, random_state=42)
    df['Anomaly'] = iso_forest.fit_predict(X)
    
    # Get anomalies (where Anomaly == -1)
    anomalies = df[df['Anomaly'] == -1].sort_values('Amount', ascending=False)
    
    print(f"\n Found {len(anomalies)} unusual expenses out of {len(df)} total!")
    
    if len(anomalies) > 0:
        print("\n Top 10 Unusual Expenses:")
        print("-"*80)
        print(f"{'Date':<12} {'Category':<15} {'Amount':<10} {'Description':<20}")
        print("-"*80)
        
        for _, row in anomalies.head(10).iterrows():
            print(f"{row['Date']:<12} {row['Category']:<15} ${row['Amount']:>8,.2f} {row['Description']:<20}")
        
        print("\n These expenses are statistically unusual based on your 24-month pattern!")
    else:
        print("\n No significant anomalies detected - your spending is very consistent!")

## Step 7: Visualizations - 2 Years of Spending Patterns

In [ ]:
if all_expenses.empty:
    print("No data to visualize yet!")
else:
    print("\n Generating visualizations for 24 months of data...\n")
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('2-Year Spending Analysis (Nov 2023 - Oct 2025)', fontsize=16, fontweight='bold')
    
    # 1. Monthly spending trend with salary
    monthly = all_expenses.groupby(['Year', 'Month'])['Amount'].sum().reset_index()
    monthly['Date'] = pd.to_datetime(monthly[['Year', 'Month']].assign(day=1))
    monthly = monthly.sort_values('Date')
    
    # Add salary data
    monthly['Salary'] = monthly.apply(
        lambda row: salary_history.get(f"{int(row['Year'])}-{int(row['Month']):02d}", 0),
        axis=1
    )
    
    ax1 = axes[0, 0]
    ax1.plot(monthly['Date'], monthly['Amount'], marker='o', linewidth=2, markersize=6, label='Spending')
    ax1.plot(monthly['Date'], monthly['Salary'], marker='s', linewidth=2, markersize=6, 
             label='Salary', linestyle='--', alpha=0.7)
    ax1.set_title('Monthly Spending vs Salary (24 Months)', fontweight='bold')
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Amount ($)')
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    ax1.legend()
    
    # 2. Seasonal comparison
    all_expenses['Season'] = all_expenses['Month'].apply(get_season)
    seasonal = all_expenses.groupby('Season')['Amount'].sum()
    
    colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']
    seasonal[['Winter', 'Spring', 'Summer', 'Autumn']].plot(
        kind='bar', ax=axes[0, 1], color=colors, width=0.7
    )
    axes[0, 1].set_title('Total Spending by Season', fontweight='bold')
    axes[0, 1].set_xlabel('Season')
    axes[0, 1].set_ylabel('Total Amount ($)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Category breakdown
    cat_totals = all_expenses.groupby('Category')['Amount'].sum().sort_values(ascending=True)
    
    cat_totals.plot(kind='barh', ax=axes[1, 0], color='#9b59b6')
    axes[1, 0].set_title('Total Spending by Category (24 Months)', fontweight='bold')
    axes[1, 0].set_xlabel('Total Amount ($)')
    axes[1, 0].set_ylabel('Category')
    
    # 4. Category spending by month (stacked area)
    cat_by_month = all_expenses.pivot_table(
        values='Amount', 
        index='Month', 
        columns='Category', 
        aggfunc='sum',
        fill_value=0
    )
    
    cat_by_month.plot(kind='area', stacked=True, ax=axes[1, 1], alpha=0.7)
    axes[1, 1].set_title('Category Spending Across Months', fontweight='bold')
    axes[1, 1].set_ylabel('Amount ($)')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_xticklabels([month_name[m] for m in cat_by_month.index], rotation=45)
    axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    
    plt.tight_layout()
    plt.show()

## Step 8: CLEVER AI Prediction - December 2025

In [ ]:
if all_expenses.empty or len(all_expenses) < 50:
    print("Need more data for predictions!")
else:
    print("\n" + "="*80)
    print(" CLEVER AI PREDICTION FOR December 2025")
    print("="*80)
    
    next_season = get_season(12)  # December
    print(f"\n Season: {next_season} (Winter)")
    print(f"\n {next_season} Tips:")
    for tip in seasonal_tips[next_season]:
        print(f"   • {tip}")
    
    print("\n" + "-"*80)
    print("CLEVER AI METHODOLOGY:")
    print("-"*80)
    print("1. TREND ANALYSIS: Detect spending increase/decrease patterns")
    print("2. YEAR-OVER-YEAR: Compare Dec 2024 vs Dec 2023 growth rate")
    print("3. INFLATION ADJUSTMENT: Apply 1% monthly compound inflation")
    print("4. SEASONAL MULTIPLIERS: Winter effects (+35% bills, +25% healthcare)")
    print("5. RECENT MOMENTUM: Weight last 3 months heavily (40%)")
    print("6. CONFIDENCE SCORING: More data = higher confidence weights")
    print("7. ADAPTIVE WEIGHTING: Adjust based on data quality and patterns")
    print("-"*80)
    
    # Calculate different prediction components
    predictions = {}
    confidence_scores = {}
    
    print("\n" + "="*80)
    print("DETAILED ANALYSIS PER CATEGORY:")
    print("="*80)
    
    total_predicted = 0
    
    for cat in categories:
        print(f"\n{cat}:")
        print("-" * 40)
        
        cat_data = all_expenses[all_expenses['Category'] == cat]
        
        # 1. Year-over-year analysis (December comparison)
        dec_2023 = cat_data[(cat_data['Year'] == 2023) & (cat_data['Month'] == 12)]['Amount'].sum()
        dec_2024 = cat_data[(cat_data['Year'] == 2024) & (cat_data['Month'] == 12)]['Amount'].sum()
        
        if dec_2023 > 0 and dec_2024 > 0:
            yoy_growth = (dec_2024 - dec_2023) / dec_2023
            # Project for 2025 with same growth
            yoy_prediction = dec_2024 * (1 + yoy_growth)
            print(f"  Dec 2023: ${dec_2023:.2f}")
            print(f"  Dec 2024: ${dec_2024:.2f} ({yoy_growth:+.1%} growth)")
            print(f"  YoY Projection 2025: ${yoy_prediction:.2f}")
            yoy_confidence = 0.85  # High confidence with 2 years data
        elif dec_2024 > 0:
            yoy_prediction = dec_2024 * 1.01  # Assume 1% growth
            print(f"  Dec 2024: ${dec_2024:.2f}")
            print(f"  YoY Projection 2025: ${yoy_prediction:.2f} (estimated)")
            yoy_confidence = 0.6  # Medium confidence with 1 year data
        else:
            yoy_prediction = 0
            yoy_confidence = 0
            print(f"  No December history available")
        
        # 2. Trend analysis (last 6 months)
        recent_6m = cat_data[
            ((cat_data['Year'] == 2025) & (cat_data['Month'] >= 6)) |
            ((cat_data['Year'] == 2024) & (cat_data['Month'] >= 11))
        ].groupby(['Year', 'Month'])['Amount'].sum().values
        
        if len(recent_6m) >= 3:
            # Calculate trend (linear regression)
            x = np.arange(len(recent_6m))
            z = np.polyfit(x, recent_6m, 1)
            trend_slope = z[0]
            trend_prediction = z[0] * len(recent_6m) + z[1]
            
            if abs(trend_slope) > 0.1:
                trend_direction = "increasing" if trend_slope > 0 else "decreasing"
                print(f"  6-month trend: {trend_direction} (${trend_slope:+.2f}/month)")
            else:
                print(f"  6-month trend: stable")
            print(f"  Trend projection: ${trend_prediction:.2f}")
            trend_confidence = 0.75
        else:
            trend_prediction = 0
            trend_confidence = 0
        
        # 3. Recent average (last 3 months)
        recent_3m = cat_data[
            (cat_data['Year'] == 2025) & 
            (cat_data['Month'].isin([9, 10, 11]))
        ].groupby('Month')['Amount'].sum()
        
        if len(recent_3m) > 0:
            recent_avg = recent_3m.mean()
            print(f"  Recent 3-month avg: ${recent_avg:.2f}")
            recent_confidence = 0.9  # High confidence - most recent data
        else:
            recent_avg = 0
            recent_confidence = 0
        
        # 4. 24-month average
        overall_avg = cat_data.groupby(['Year', 'Month'])['Amount'].sum().mean()
        print(f"  24-month average: ${overall_avg:.2f}")
        
        # 5. Apply seasonal multiplier
        seasonal_mult = get_seasonal_multiplier(cat, next_season)
        if seasonal_mult != 1.0:
            print(f"  Winter multiplier: {seasonal_mult:.2f}x")
        
        # 6. Apply inflation (1 month ahead)
        inflation_factor = 1.01
        
        # 7. CLEVER WEIGHTING based on data quality
        weights = []
        values = []
        
        # YoY prediction (if available)
        if yoy_confidence > 0:
            weights.append(0.30 * yoy_confidence)
            values.append(yoy_prediction)
        
        # Trend prediction (if available)
        if trend_confidence > 0 and trend_prediction > 0:
            weights.append(0.20 * trend_confidence)
            values.append(trend_prediction)
        
        # Recent average (heavily weighted)
        if recent_confidence > 0 and recent_avg > 0:
            weights.append(0.40 * recent_confidence)
            values.append(recent_avg * seasonal_mult)
        
        # Overall average (baseline)
        weights.append(0.10)
        values.append(overall_avg * seasonal_mult)
        
        # Normalize weights
        total_weight = sum(weights)
        weights = [w / total_weight for w in weights]
        
        # Calculate weighted prediction
        predicted = sum(w * v for w, v in zip(weights, values))
        
        # Apply inflation
        predicted *= inflation_factor
        
        # Calculate confidence score
        confidence = min(100, (yoy_confidence * 30 + trend_confidence * 20 + 
                               recent_confidence * 40 + 10))
        
        predictions[cat] = predicted
        confidence_scores[cat] = confidence
        total_predicted += predicted
        
        print(f"  PREDICTED: ${predicted:.2f} (confidence: {confidence:.0f}%)")
    
    # Summary table
    print("\n" + "="*80)
    print("PREDICTION SUMMARY:")
    print("="*80)
    print(f"{'Category':<15} {'Predicted':<12} {'Budget':<12} {'Confidence':<12} {'Advice':<20}")
    print("-"*80)
    
    for cat in categories:
        predicted = predictions[cat]
        budget = budgets[cat]
        confidence = confidence_scores[cat]
        
        if budget > 0 and predicted > 0:
            diff = predicted - budget
            diff_pct = (diff / budget) * 100
            
            if diff_pct > 15:
                advice = f"Increase ${diff:.0f}"
            elif diff_pct < -15:
                advice = f"Reduce ${abs(diff):.0f}"
            else:
                advice = "Keep current"
        else:
            advice = "Set budget"
        
        print(f"{cat:<15} ${predicted:>9,.2f}  ${budget:>9,.2f}  {confidence:>6.0f}%      {advice:<20}")
    
    print("-"*80)
    print(f"{'TOTAL:':<15} ${total_predicted:>9,.2f}")
    
    if monthly_salary > 0:
        print(f"\n Your Salary: ${monthly_salary:,.2f}")
        savings = monthly_salary - total_predicted
        
        if savings > 0:
            savings_pct = (savings / monthly_salary) * 100
            print(f" Predicted Savings: ${savings:,.2f} ({savings_pct:.0f}% of salary)")
        else:
            print(f" WARNING: Over budget by ${abs(savings):,.2f}!")
    
    # Insights
    print("\n" + "="*80)
    print(" KEY INSIGHTS")
    print("="*80)
    
    # Compare winter to other seasons
    all_expenses['Season'] = all_expenses['Month'].apply(get_season)
    season_avgs = all_expenses.groupby('Season')['Amount'].sum() / all_expenses.groupby('Season')['Month'].nunique()
    
    winter_avg = season_avgs.get('Winter', 0)
    summer_avg = season_avgs.get('Summer', 0)
    
    if winter_avg > 0 and summer_avg > 0:
        diff = winter_avg - summer_avg
        diff_pct = (diff / summer_avg) * 100
        print(f"\n Your Winter spending averages ${diff:.2f} ({diff_pct:+.1f}%) more than Summer")
    
    # Show categories with highest confidence
    high_conf_cats = [cat for cat, conf in confidence_scores.items() if conf >= 80]
    if high_conf_cats:
        print(f"\n High confidence predictions (80%+): {', '.join(high_conf_cats)}")
    
    # Show categories needing attention
    attention_cats = []
    for cat in categories:
        if budgets[cat] > 0 and predictions[cat] > budgets[cat] * 1.15:
            attention_cats.append(cat)
    
    if attention_cats:
        print(f"\n Categories needing budget increase: {', '.join(attention_cats)}")
    
    # Expected Winter increases
    print("\n Expected December (Winter) changes:")
    winter_impacts = []
    for cat in categories:
        mult = get_seasonal_multiplier(cat, 'Winter')
        if mult > 1.1:
            change_pct = (mult - 1.0) * 100
            winter_impacts.append(f"   {cat}: +{change_pct:.0f}%")
        elif mult < 0.9:
            change_pct = (1.0 - mult) * 100
            winter_impacts.append(f"   {cat}: -{change_pct:.0f}%")
    
    for impact in winter_impacts:
        print(impact)
    
    print("\n" + "="*80)
    print(" This CLEVER AI uses 7 different analytical methods!")
    print(" Prediction accuracy improves with 2 years of seasonal data.")
    print(" Confidence scores show reliability of each category prediction.")
    print("="*80)

## Step 9: Financial Goals - Track Your Savings Goals

In [ ]:
print("\n" + "="*80)
print(" 🎯 FINANCIAL GOALS MANAGER")
print("="*80)

if monthly_salary == 0:
    print("\n⚠️ Please run Step 2 first to set your salary!")
else:
    while True:
        print("\nWhat would you like to do?")
        print("   1. Add a new financial goal")
        print("   2. View existing goals and progress")
        print("   3. View detailed goal projections (single goal)")
        if len(financial_goals) >= 2:
            print("   4. Compare savings strategies (Sequential vs Parallel)")
        print("   0. Exit (go to next section)")
        
        choice = input(f"\nSelect option (0-{4 if len(financial_goals) >= 2 else 3}): ").strip()
        
        if choice == '0':
            print("\n✓ Exiting Financial Goals Manager...")
            break
        
        elif choice == '1':
            # ADD NEW GOAL
            print("\n" + "-"*80)
            print("📝 ADD NEW FINANCIAL GOAL")
            print("-"*80)
            
            goal_name = input("\nGoal name (e.g., 'New Laptop', 'Vacation'): ").strip()
            
            if not goal_name:
                print("❌ Goal name cannot be empty!")
            else:
                try:
                    target_amount = float(input("Target amount ($): "))
                    
                    if target_amount <= 0:
                        print("❌ Amount must be positive!")
                    else:
                        print("\nTarget date (when do you want to achieve this goal?)")
                        target_year = int(input("  Year (e.g., 2026): "))
                        target_month = int(input("  Month (1-12): "))
                        
                        if target_month < 1 or target_month > 12:
                            print("❌ Invalid month!")
                        else:
                            # Calculate months until goal
                            current_date = 2025 * 12 + 12  # December 2025
                            target_date = target_year * 12 + target_month
                            months_until_goal = target_date - current_date
                            
                            if months_until_goal <= 0:
                                print("❌ Target date must be in the future!")
                            else:
                                # Calculate average monthly savings needed
                                monthly_savings_needed = target_amount / months_until_goal
                                
                                # Calculate current average monthly savings
                                recent_months = all_expenses[
                                    (all_expenses['Year'] == 2025) & 
                                    (all_expenses['Month'].isin([9, 10, 11]))
                                ]
                                
                                if len(recent_months) > 0:
                                    recent_spending = recent_months.groupby('Month')['Amount'].sum().mean()
                                    avg_monthly_savings = monthly_salary - recent_spending
                                else:
                                    total_budget = sum(budgets.values())
                                    avg_monthly_savings = monthly_salary - total_budget
                                
                                # Store goal
                                goal = {
                                    'name': goal_name,
                                    'target_amount': target_amount,
                                    'target_year': target_year,
                                    'target_month': target_month,
                                    'months_until_goal': months_until_goal,
                                    'monthly_savings_needed': monthly_savings_needed,
                                    'created_date': '2025-12'
                                }
                                financial_goals.append(goal)
                                
                                # Show summary
                                print("\n" + "="*80)
                                print(f"✓ Goal Added: {goal_name}")
                                print("="*80)
                                print(f"Target Amount:    ${target_amount:,.2f}")
                                print(f"Target Date:      {month_name[target_month]} {target_year}")
                                print(f"Months to Save:   {months_until_goal} months")
                                print(f"Monthly Savings Needed: ${monthly_savings_needed:.2f}")
                                print(f"\nYour Current Average Monthly Savings: ${avg_monthly_savings:.2f}")
                                
                                if avg_monthly_savings >= monthly_savings_needed:
                                    surplus = avg_monthly_savings - monthly_savings_needed
                                    print(f"✓ You're on track! Surplus: ${surplus:.2f}/month")
                                else:
                                    deficit = monthly_savings_needed - avg_monthly_savings
                                    print(f"⚠️  You need to save ${deficit:.2f} more per month")
                                    print(f"   Suggested actions:")
                                    print(f"   • Reduce spending by ${deficit:.2f}/month")
                                    print(f"   • Or extend goal to {months_until_goal + int(deficit/avg_monthly_savings) + 1} months")
                                
                                print("="*80)
                                
                except ValueError:
                    print("❌ Invalid input! Please enter valid numbers.")
        
        elif choice == '2':
            # VIEW GOALS
            print("\n" + "-"*80)
            print("📊 YOUR FINANCIAL GOALS")
            print("-"*80)
            
            if not financial_goals:
                print("\nℹ️  No goals yet! Select option 1 to add your first goal.")
            else:
                # Calculate current average monthly savings
                recent_months = all_expenses[
                    (all_expenses['Year'] == 2025) & 
                    (all_expenses['Month'].isin([9, 10, 11]))
                ]
                
                if len(recent_months) > 0:
                    recent_spending = recent_months.groupby('Month')['Amount'].sum().mean()
                    avg_monthly_savings = monthly_salary - recent_spending
                else:
                    total_budget = sum(budgets.values())
                    avg_monthly_savings = monthly_salary - total_budget
                
                print(f"\nYour Monthly Salary: ${monthly_salary:,.2f}")
                print(f"Average Monthly Savings: ${avg_monthly_savings:.2f}")
                print("\n" + "="*80)
                
                for i, goal in enumerate(financial_goals, 1):
                    print(f"\n🎯 Goal #{i}: {goal['name']}")
                    print("-"*80)
                    print(f"Target Amount:     ${goal['target_amount']:,.2f}")
                    print(f"Target Date:       {month_name[goal['target_month']]} {goal['target_year']}")
                    print(f"Months Remaining:  {goal['months_until_goal']} months")
                    print(f"Need to Save:      ${goal['monthly_savings_needed']:.2f}/month")
                    
                    # Calculate progress
                    progress = avg_monthly_savings * goal['months_until_goal']
                    progress_pct = min(100, (progress / goal['target_amount']) * 100)
                    
                    # Status
                    if avg_monthly_savings >= goal['monthly_savings_needed']:
                        status = "✓ On Track"
                        completion_date = f"{month_name[goal['target_month']]} {goal['target_year']}"
                    else:
                        status = "⚠️  Behind"
                        if avg_monthly_savings > 0:
                            actual_months = goal['target_amount'] / avg_monthly_savings
                            actual_date = (2025 * 12 + 12) + int(actual_months)
                            actual_year = actual_date // 12
                            actual_month = (actual_date % 12)
                            if actual_month == 0:
                                actual_month = 12
                                actual_year -= 1
                            completion_date = f"{month_name[actual_month]} {actual_year}"
                        else:
                            completion_date = "Cannot reach (no savings)"
                    
                    print(f"Status:            {status}")
                    print(f"Projected Completion: {completion_date}")
                    
                    # Progress bar
                    bar_length = 40
                    filled = int(bar_length * progress_pct / 100)
                    bar = '█' * filled + '░' * (bar_length - filled)
                    print(f"Progress:          [{bar}] {progress_pct:.1f}%")
                    
                    print("="*80)
        
        elif choice == '3':
            # DETAILED PROJECTIONS - SINGLE GOAL
            print("\n" + "-"*80)
            print("📈 DETAILED GOAL PROJECTIONS")
            print("-"*80)
            
            if not financial_goals:
                print("\nℹ️  No goals yet! Select option 1 to add your first goal.")
            else:
                # Select goal
                print("\nYour goals:")
                for i, goal in enumerate(financial_goals, 1):
                    print(f"   {i}. {goal['name']} - ${goal['target_amount']:,.2f} by {month_name[goal['target_month']]} {goal['target_year']}")
                
                goal_choice = input("\nSelect goal number: ").strip()
                
                try:
                    goal_idx = int(goal_choice) - 1
                    if goal_idx < 0 or goal_idx >= len(financial_goals):
                        print("❌ Invalid goal number!")
                    else:
                        goal = financial_goals[goal_idx]
                        
                        print("\n" + "="*80)
                        print(f"🎯 PROJECTION FOR: {goal['name']}")
                        print("="*80)
                        
                        # Calculate month-by-month projection
                        print(f"\nTarget: ${goal['target_amount']:,.2f} by {month_name[goal['target_month']]} {goal['target_year']}")
                        print(f"Monthly Savings Needed: ${goal['monthly_savings_needed']:.2f}")
                        print("\n" + "-"*80)
                        print("MONTH-BY-MONTH SAVINGS PROJECTION:")
                        print("-"*80)
                        print(f"{'Month':<15} {'Salary':<12} {'Est. Spending':<15} {'Savings':<12} {'Total Saved':<15}")
                        print("-"*80)
                        
                        current_month_num = 12  # December
                        current_year_num = 2025
                        total_saved = 0
                        
                        # Calculate REALISTIC average spending (use budgets as baseline, not historical)
                        recent_months = all_expenses[
                            (all_expenses['Year'] == 2025) & 
                            (all_expenses['Month'].isin([9, 10, 11]))
                        ]
                        
                        if len(recent_months) > 0:
                            historical_spending = recent_months.groupby('Month')['Amount'].sum().mean()
                        else:
                            historical_spending = sum(budgets.values())
                        
                        # Use budgets as realistic baseline (don't let spending exceed salary)
                        baseline_spending = min(historical_spending, sum(budgets.values()), monthly_salary * 0.85)
                        
                        for month_offset in range(goal['months_until_goal'] + 1):
                            # Calculate month/year
                            total_months = current_month_num + month_offset
                            month = ((total_months - 1) % 12) + 1
                            year = current_year_num + (total_months - 1) // 12
                            
                            # Get season and apply MODERATE multiplier to spending
                            season = get_season(month)
                            
                            # Calculate projected spending with CAPPED seasonal variation
                            season_avg_mult = np.mean([
                                get_seasonal_multiplier(cat, season) 
                                for cat in categories
                            ])
                            
                            # Cap multiplier to reasonable range (0.9 to 1.15)
                            season_avg_mult = min(1.15, max(0.9, season_avg_mult))
                            
                            projected_spending = baseline_spending * season_avg_mult
                            
                            # CRITICAL: Never let spending exceed 85% of salary
                            projected_spending = min(projected_spending, monthly_salary * 0.85)
                            
                            # Calculate savings for this month
                            monthly_savings = monthly_salary - projected_spending
                            total_saved += monthly_savings
                            
                            month_str = f"{month_name[month]} {year}"
                            
                            # Status indicator
                            if total_saved >= goal['target_amount']:
                                status = "✓"
                            elif monthly_savings >= goal['monthly_savings_needed']:
                                status = "→"
                            else:
                                status = "⚠️ "
                            
                            print(f"{status} {month_str:<13} ${monthly_salary:>9,.2f}  ${projected_spending:>12,.2f}  ${monthly_savings:>10,.2f}  ${total_saved:>12,.2f}")
                            
                            # Check if goal reached
                            if total_saved >= goal['target_amount'] and month_offset > 0:
                                print("-"*80)
                                print(f"🎉 GOAL ACHIEVED in {month_str}!")
                                months_early = goal['months_until_goal'] - month_offset
                                if months_early > 0:
                                    print(f"✓ {months_early} months ahead of schedule!")
                                break
                        else:
                            # Loop completed without breaking (goal not reached)
                            if total_saved < goal['target_amount']:
                                shortage = goal['target_amount'] - total_saved
                                print("-"*80)
                                print(f"⚠️  Short by ${shortage:.2f}")
                                if monthly_savings > 0:
                                    extra_months = int(shortage / monthly_savings) + 1
                                    print(f"   Need {extra_months} more months to reach goal")
                                else:
                                    print(f"   Cannot reach goal with current spending level")
                        
                        print("="*80)
                        
                        # Recommendations
                        print("\n💡 RECOMMENDATIONS:")
                        print("-"*80)
                        
                        avg_monthly_savings = monthly_salary - baseline_spending
                        
                        if avg_monthly_savings < goal['monthly_savings_needed']:
                            deficit = goal['monthly_savings_needed'] - avg_monthly_savings
                            print(f"• Reduce monthly spending by ${deficit:.2f}")
                            print("  Suggested categories to cut:")
                            
                            # Find categories with highest spending
                            if len(all_expenses) > 0:
                                cat_spending = all_expenses.groupby('Category')['Amount'].sum().sort_values(ascending=False)
                                for cat in cat_spending.head(3).index:
                                    reduction = deficit / 3
                                    print(f"    - {cat}: reduce by ${reduction:.2f}/month")
                            else:
                                print(f"    - Review your budget and cut non-essential expenses")
                        else:
                            surplus = avg_monthly_savings - goal['monthly_savings_needed']
                            print(f"✓ You're on track!")
                            print(f"• Extra savings per month: ${surplus:.2f}")
                            print(f"• Consider setting another goal with your surplus")
                        
                        print("="*80)
                        
                except ValueError:
                    print("❌ Invalid input!")
        
        elif choice == '4' and len(financial_goals) >= 2:
            # MULTI-GOAL STRATEGY COMPARISON
            print("\n" + "="*80)
            print("💰 MULTI-GOAL SAVINGS STRATEGY COMPARISON")
            print("="*80)
            
            # Sort goals by target date
            sorted_goals = sorted(financial_goals, key=lambda g: (g['target_year'], g['target_month']))
            
            print("\nYour Goals (sorted by target date):")
            for i, goal in enumerate(sorted_goals, 1):
                print(f"   {i}. {goal['name']} - ${goal['target_amount']:,.2f} by {month_name[goal['target_month']]} {goal['target_year']}")
                print(f"      Need: ${goal['monthly_savings_needed']:.2f}/month for {goal['months_until_goal']} months")
            
            # Calculate available savings
            recent_months = all_expenses[
                (all_expenses['Year'] == 2025) & 
                (all_expenses['Month'].isin([9, 10, 11]))
            ]
            
            if len(recent_months) > 0:
                recent_spending = recent_months.groupby('Month')['Amount'].sum().mean()
                avg_monthly_savings = monthly_salary - recent_spending
            else:
                total_budget = sum(budgets.values())
                avg_monthly_savings = monthly_salary - total_budget
            
            print(f"\nYour Monthly Salary: ${monthly_salary:,.2f}")
            print(f"Average Monthly Savings: ${avg_monthly_savings:.2f}")
            
            # OPTION A: SEQUENTIAL (one at a time)
            print("\n" + "="*80)
            print("📊 OPTION A: SEQUENTIAL SAVINGS (One Goal at a Time)")
            print("="*80)
            print("Save for each goal fully before starting the next one\n")
            
            current_month = 2025 * 12 + 12  # December 2025
            sequential_timeline = []
            
            for goal in sorted_goals:
                months_needed = int(np.ceil(goal['target_amount'] / avg_monthly_savings)) if avg_monthly_savings > 0 else 999
                completion_month = current_month + months_needed
                completion_year = completion_month // 12
                completion_month_num = completion_month % 12
                if completion_month_num == 0:
                    completion_month_num = 12
                    completion_year -= 1
                
                target_month_total = goal['target_year'] * 12 + goal['target_month']
                delay_months = completion_month - target_month_total
                
                sequential_timeline.append({
                    'name': goal['name'],
                    'target_amount': goal['target_amount'],
                    'target_date': f"{month_name[goal['target_month']]} {goal['target_year']}",
                    'completion_date': f"{month_name[completion_month_num]} {completion_year}",
                    'months_needed': months_needed,
                    'delay_months': delay_months,
                    'status': '✓ On time' if delay_months <= 0 else f'⚠️  {delay_months} months late'
                })
                
                current_month = completion_month
            
            print(f"{'Goal':<20} {'Target':<12} {'Target Date':<15} {'Complete By':<15} {'Status':<20}")
            print("-"*100)
            for item in sequential_timeline:
                print(f"{item['name']:<20} ${item['target_amount']:>9,.2f}  {item['target_date']:<15} {item['completion_date']:<15} {item['status']}")
            
            total_seq_months = sum(item['months_needed'] for item in sequential_timeline)
            print(f"\n   Total Time: {total_seq_months} months")
            
            # OPTION B: PARALLEL (all at once)
            print("\n" + "="*80)
            print("📊 OPTION B: PARALLEL SAVINGS (All Goals Simultaneously)")
            print("="*80)
            print("Save for all goals at the same time with split contributions\n")
            
            total_monthly_needed = sum(g['monthly_savings_needed'] for g in sorted_goals)
            
            print(f"Total Monthly Savings Needed: ${total_monthly_needed:.2f}")
            print(f"Your Monthly Savings Available: ${avg_monthly_savings:.2f}")
            
            if avg_monthly_savings >= total_monthly_needed:
                print(f"✓ You CAN save for all goals in parallel!\n")
                surplus = avg_monthly_savings - total_monthly_needed
                print(f"Extra surplus: ${surplus:.2f}/month\n")
                
                parallel_timeline = []
                for goal in sorted_goals:
                    target_month_total = goal['target_year'] * 12 + goal['target_month']
                    delay_months = (2025 * 12 + 12) + goal['months_until_goal'] - target_month_total
                    
                    parallel_timeline.append({
                        'name': goal['name'],
                        'target_amount': goal['target_amount'],
                        'monthly_contribution': goal['monthly_savings_needed'],
                        'target_date': f"{month_name[goal['target_month']]} {goal['target_year']}",
                        'months_needed': goal['months_until_goal'],
                        'status': '✓ On time'
                    })
                
                print(f"{'Goal':<20} {'Target':<12} {'Monthly Save':<15} {'Target Date':<15} {'Status':<20}")
                print("-"*100)
                for item in parallel_timeline:
                    print(f"{item['name']:<20} ${item['target_amount']:>9,.2f}  ${item['monthly_contribution']:>12.2f}  {item['target_date']:<15} {item['status']}")
                
                max_parallel_months = max(g['months_until_goal'] for g in sorted_goals)
                print(f"\n   Total Time: {max_parallel_months} months (all goals complete)")
                
            else:
                deficit = total_monthly_needed - avg_monthly_savings
                print(f"⚠️  You're SHORT by ${deficit:.2f}/month\n")
                print(f"To save in parallel, you need to:")
                print(f"   • Reduce spending by ${deficit:.2f}/month, OR")
                print(f"   • Extend goal timelines, OR")
                print(f"   • Use Sequential strategy (Option A)\n")
                
                # Show what you CAN afford in parallel
                print("What you CAN afford with parallel savings:")
                print(f"{'Goal':<20} {'Target':<12} {'Monthly Save':<15} {'Affordable?':<15}")
                print("-"*80)
                
                remaining_budget = avg_monthly_savings
                for goal in sorted_goals:
                    if remaining_budget >= goal['monthly_savings_needed']:
                        status = f"✓ Yes (${goal['monthly_savings_needed']:.2f}/mo)"
                        remaining_budget -= goal['monthly_savings_needed']
                    else:
                        status = f"✗ No (need ${goal['monthly_savings_needed']:.2f}, have ${remaining_budget:.2f})"
                    
                    print(f"{goal['name']:<20} ${goal['target_amount']:>9,.2f}  ${goal['monthly_savings_needed']:>12.2f}  {status}")
            
            # RECOMMENDATION
            print("\n" + "="*80)
            print("💡 RECOMMENDATION:")
            print("="*80)
            
            if avg_monthly_savings >= total_monthly_needed:
                time_saved = total_seq_months - max(g['months_until_goal'] for g in sorted_goals)
                print(f"✓ Choose PARALLEL (Option B)")
                print(f"  • Achieves all goals {time_saved} months faster")
                print(f"  • All goals completed by their target dates")
                print(f"  • Better financial discipline (consistent saving)")
            else:
                print(f"⚠️  Choose SEQUENTIAL (Option A) or reduce spending by ${deficit:.2f}/month")
                print(f"  • Sequential is more manageable with current income")
                print(f"  • Focus on one goal at a time")
                print(f"  • Consider prioritizing most urgent goal first")
            
            print("="*80)
        
        else:
            print(f"\n❌ Invalid option! Please select 0-{4 if len(financial_goals) >= 2 else 3}.")


## Step 10: Smart Goal Optimizer - AI-Powered Spending Reduction Plan

In [ ]:
from sklearn.ensemble import IsolationForest

print("\n" + "="*80)
print(" 🎯 SMART GOAL OPTIMIZER - AI-Powered Reduction Plan")
print("="*80)

if not financial_goals or len(all_expenses) < 50 or monthly_salary == 0:
    print("\n⚠️  Requirements:")
    if monthly_salary == 0:
        print("   • Please run Step 2 to set your salary")
    if len(all_expenses) < 50:
        print("   • Need more expense data (run Step 2 or import CSV)")
    if not financial_goals:
        print("   • Please add a financial goal in Step 9")
else:
    print("\nThis AI will analyze your spending patterns and create a personalized")
    print("reduction plan to help you achieve your financial goals faster!")
    
    # Select optimization mode
    print("\n" + "-"*80)
    print("Your Financial Goals:")
    print("-"*80)
    for i, goal in enumerate(financial_goals, 1):
        print(f"   {i}. {goal['name']} - ${goal['target_amount']:,.2f} by {month_name[goal['target_month']]} {goal['target_year']}")
    
    if len(financial_goals) > 1:
        print(f"   {len(financial_goals) + 1}. Optimize for ALL goals (combined)")
    
    goal_choice = input(f"\nSelect goal to optimize for (1-{len(financial_goals) + (1 if len(financial_goals) > 1 else 0)}): ").strip()
    
    try:
        goal_idx = int(goal_choice) - 1
        
        # Check if user selected "ALL goals" option
        optimize_all = (len(financial_goals) > 1 and goal_idx == len(financial_goals))
        
        if goal_idx < 0 or goal_idx > len(financial_goals):
            print("❌ Invalid goal number!")
        else:
            if optimize_all:
                # OPTIMIZE FOR ALL GOALS
                print("\n" + "="*80)
                print(" OPTIMIZATION PLAN FOR: ALL GOALS (COMBINED)")
                print("="*80)
                
                # Calculate total savings needed for all goals
                total_savings_needed = sum(g['monthly_savings_needed'] for g in financial_goals)
                
                # Calculate current savings rate
                recent_months = all_expenses[
                    (all_expenses['Year'] == 2025) & 
                    (all_expenses['Month'].isin([9, 10, 11]))
                ]
                
                if len(recent_months) > 0:
                    recent_spending = recent_months.groupby('Month')['Amount'].sum().mean()
                    current_savings = monthly_salary - recent_spending
                else:
                    recent_spending = sum(budgets.values())
                    current_savings = monthly_salary - recent_spending
                
                savings_gap = total_savings_needed - current_savings
                
                print(f"\n📊 Current Situation:")
                print(f"   Monthly Salary:        ${monthly_salary:,.2f}")
                print(f"   Current Spending:      ${recent_spending:,.2f}")
                print(f"   Current Savings:       ${current_savings:,.2f}/month")
                print(f"\n🎯 Combined Goals:")
                for i, goal in enumerate(financial_goals, 1):
                    print(f"   {i}. {goal['name']}: ${goal['monthly_savings_needed']:.2f}/month")
                print(f"\n   Total Savings Needed:  ${total_savings_needed:.2f}/month")
                
                if savings_gap > 0:
                    print(f"\n⚠️  Savings Gap: ${savings_gap:.2f}/month")
                    print(f"   You need to save ${savings_gap:.2f} more per month to reach ALL goals!")
                else:
                    print(f"\n✓ Great! You're already saving enough for all goals (${abs(savings_gap):.2f} surplus)")
                    savings_gap = 0
                
            else:
                # OPTIMIZE FOR SINGLE GOAL
                goal = financial_goals[goal_idx]
                
                print("\n" + "="*80)
                print(f" OPTIMIZATION PLAN FOR: {goal['name']}")
                print("="*80)
                
                # Calculate current savings rate
                recent_months = all_expenses[
                    (all_expenses['Year'] == 2025) & 
                    (all_expenses['Month'].isin([9, 10, 11]))
                ]
                
                if len(recent_months) > 0:
                    recent_spending = recent_months.groupby('Month')['Amount'].sum().mean()
                    current_savings = monthly_salary - recent_spending
                else:
                    recent_spending = sum(budgets.values())
                    current_savings = monthly_salary - recent_spending
                
                savings_gap = goal['monthly_savings_needed'] - current_savings
                
                print(f"\n📊 Current Situation:")
                print(f"   Monthly Salary:        ${monthly_salary:,.2f}")
                print(f"   Current Spending:      ${recent_spending:,.2f}")
                print(f"   Current Savings:       ${current_savings:,.2f}/month")
                print(f"   Target Savings Needed: ${goal['monthly_savings_needed']:.2f}/month")
                
                if savings_gap > 0:
                    print(f"\n⚠️  Savings Gap: ${savings_gap:.2f}/month")
                    print(f"   You need to save ${savings_gap:.2f} more per month to reach your goal!")
                else:
                    print(f"\n✓ Great! You're already saving enough (${abs(savings_gap):.2f} surplus)")
                    savings_gap = 0
            
            # Analyze spending by category (COMMON FOR BOTH)
            print("\n" + "="*80)
            print(" 🔍 AI SPENDING ANALYSIS")
            print("="*80)
            
            # Get category spending (last 6 months)
            last_6m = all_expenses[
                ((all_expenses['Year'] == 2025) & (all_expenses['Month'] >= 6)) |
                ((all_expenses['Year'] == 2024) & (all_expenses['Month'] >= 11))
            ]
            
            category_analysis = []
            
            for cat in categories:
                cat_data = last_6m[last_6m['Category'] == cat]
                
                if len(cat_data) > 0:
                    # Monthly average
                    monthly_avg = cat_data.groupby(['Year', 'Month'])['Amount'].sum().mean()
                    
                    # Percentage of total spending
                    pct_of_spending = (monthly_avg / recent_spending) * 100
                    
                    # Detect anomalies in this category
                    if len(cat_data) >= 10:
                        cat_amounts = cat_data['Amount'].values.reshape(-1, 1)
                        iso = IsolationForest(contamination=0.1, random_state=42)
                        anomaly_labels = iso.fit_predict(cat_amounts)
                        num_anomalies = (anomaly_labels == -1).sum()
                        anomaly_pct = (num_anomalies / len(cat_data)) * 100
                    else:
                        num_anomalies = 0
                        anomaly_pct = 0
                    
                    # Calculate reduction potential (higher % = more potential)
                    # Factors: high spending %, high anomalies, non-essential categories
                    essential_categories = ['Bills', 'Healthcare']
                    reduction_score = pct_of_spending
                    
                    if cat not in essential_categories:
                        reduction_score *= 1.5  # Boost non-essential
                    
                    if anomaly_pct > 5:
                        reduction_score *= 1.2  # Boost if many anomalies
                    
                    category_analysis.append({
                        'category': cat,
                        'monthly_avg': monthly_avg,
                        'pct_of_spending': pct_of_spending,
                        'num_anomalies': num_anomalies,
                        'anomaly_pct': anomaly_pct,
                        'reduction_score': reduction_score,
                        'is_essential': cat in essential_categories
                    })
            
            # Sort by reduction score
            category_analysis.sort(key=lambda x: x['reduction_score'], reverse=True)
            
            # Display analysis
            print(f"\n{'Category':<15} {'Avg/Month':<12} {'% Spending':<12} {'Anomalies':<12} {'Priority':<10}")
            print("-"*80)
            
            for item in category_analysis:
                priority = "🔴 High" if item['reduction_score'] > 20 else ("🟡 Medium" if item['reduction_score'] > 10 else "🟢 Low")
                print(f"{item['category']:<15} ${item['monthly_avg']:>9,.2f}  {item['pct_of_spending']:>9.1f}%   {item['num_anomalies']:>6} ({item['anomaly_pct']:.1f}%)  {priority}")
            
            # Generate reduction plan
            if savings_gap > 0:
                print("\n" + "="*80)
                print(" 💡 PERSONALIZED REDUCTION PLAN")
                print("="*80)
                
                if optimize_all:
                    print(f"\nGoal: Reduce spending by ${savings_gap:.2f}/month to reach ALL goals simultaneously\n")
                    print("Goals breakdown:")
                    for i, g in enumerate(financial_goals, 1):
                        print(f"   {i}. {g['name']}: ${g['monthly_savings_needed']:.2f}/month → {month_name[g['target_month']]} {g['target_year']}")
                    print()
                else:
                    print(f"\nGoal: Reduce spending by ${savings_gap:.2f}/month to reach '{goal['name']}'\n")
                
                # Category-specific advice
                reduction_tips = {
                    'Food': [
                        "Cook at home more often instead of eating out",
                        "Reduce friend gatherings at expensive restaurants",
                        "Buy generic brands instead of premium products",
                        "Plan meals weekly to avoid impulse purchases",
                        "Pack lunch for work/school"
                    ],
                    'Transportation': [
                        "Use public transport instead of taxis",
                        "Carpool with colleagues or friends",
                        "Walk or bike for short distances",
                        "Combine errands to reduce fuel costs",
                        "Consider monthly transit passes"
                    ],
                    'Entertainment': [
                        "Limit streaming subscriptions to 1-2 services",
                        "Look for free community events",
                        "Have movie nights at home instead of theaters",
                        "Reduce gaming purchases - wait for sales",
                        "Explore free hobbies like hiking or reading"
                    ],
                    'Shopping': [
                        "Wait 24 hours before non-essential purchases",
                        "Buy clothes only when necessary",
                        "Shop during sales and use coupons",
                        "Avoid impulse online shopping",
                        "Sell items you no longer need"
                    ],
                    'Bills': [
                        "Negotiate better rates with providers",
                        "Reduce energy usage (turn off lights/AC)",
                        "Bundle internet/phone services for discounts",
                        "Consider cheaper phone plans",
                        "Monitor and fix water leaks"
                    ],
                    'Healthcare': [
                        "Use generic medications when possible",
                        "Take advantage of preventive care",
                        "Compare prices at different pharmacies",
                        "Maintain healthy lifestyle to reduce doctor visits",
                        "Check if employer offers health benefits"
                    ],
                    'Education': [
                        "Use free online resources (YouTube, Khan Academy)",
                        "Borrow books from library instead of buying",
                        "Look for student discounts",
                        "Share textbooks with classmates",
                        "Apply for scholarships or grants"
                    ],
                    'Other': [
                        "Set a gift budget and stick to it",
                        "Make handmade gifts instead of buying",
                        "Cancel unused subscriptions",
                        "Reduce charitable donations temporarily",
                        "DIY repairs when possible"
                    ]
                }
                
                # Distribute reduction across top categories
                remaining_gap = savings_gap
                recommendations = []
                
                for item in category_analysis:
                    if remaining_gap <= 0:
                        break
                    
                    cat = item['category']
                    current_spending = item['monthly_avg']
                    
                    # Calculate potential reduction (10-25% depending on priority)
                    if item['is_essential']:
                        reduction_pct = 0.10  # Only 10% for essentials
                    else:
                        reduction_pct = min(0.25, 0.15 + (item['anomaly_pct'] / 100))
                    
                    potential_reduction = current_spending * reduction_pct
                    actual_reduction = min(potential_reduction, remaining_gap)
                    
                    if actual_reduction > 1:  # Only show if >$1
                        new_spending = current_spending - actual_reduction
                        recommendations.append({
                            'category': cat,
                            'current': current_spending,
                            'target': new_spending,
                            'reduction': actual_reduction,
                            'pct': (actual_reduction / current_spending) * 100
                        })
                        remaining_gap -= actual_reduction
                
                # Display recommendations
                print(f"{'Category':<15} {'Current':<12} {'→ Target':<12} {'Reduce':<12} {'Action'}")
                print("-"*80)
                
                for rec in recommendations:
                    cat = rec['category']
                    print(f"{cat:<15} ${rec['current']:>9,.2f}  ${rec['target']:>9,.2f}  -${rec['reduction']:>8.2f} ({rec['pct']:.0f}%)")
                    
                    # Show 2-3 specific tips
                    tips = reduction_tips.get(cat, [])[:3]
                    for tip in tips:
                        print(f"   • {tip}")
                    print()
                
                total_reduction = sum(r['reduction'] for r in recommendations)
                
                print("="*80)
                print(f"📊 SUMMARY:")
                print(f"   Total Monthly Reduction: ${total_reduction:.2f}")
                print(f"   New Monthly Spending:    ${recent_spending - total_reduction:.2f}")
                print(f"   New Monthly Savings:     ${current_savings + total_reduction:.2f}")
                
                if total_reduction >= savings_gap:
                    print(f"\n✅ SUCCESS! This plan will help you reach your goal{'s' if optimize_all else ''}!")
                    
                    if optimize_all:
                        print(f"   With ${current_savings + total_reduction:.2f}/month, you can save for:")
                        for g in financial_goals:
                            print(f"   • {g['name']}: ${g['monthly_savings_needed']:.2f}/month ✓")
                        surplus = (current_savings + total_reduction) - total_savings_needed
                        if surplus > 0:
                            print(f"   • Plus ${surplus:.2f}/month extra!")
                    else:
                        months_to_goal = goal['target_amount'] / (current_savings + total_reduction)
                        print(f"   Estimated completion: {int(months_to_goal)} months")
                else:
                    shortfall = savings_gap - total_reduction
                    print(f"\n⚠️  Still short by ${shortfall:.2f}/month")
                    print(f"   Consider extending your goal timeline or finding additional income")
                
                print("\n💪 Track your progress monthly and adjust as needed!")
                print("="*80)
            else:
                if optimize_all:
                    print("\n✅ You're already saving enough for ALL goals! Keep up the good work!")
                    print(f"   Extra surplus: ${abs(savings_gap):.2f}/month")
                    print("   Consider building an emergency fund or investing the surplus.")
                else:
                    print("\n✅ You're already on track! Keep up the good work!")
                    print("   Consider setting a more ambitious goal or building an emergency fund.")
    
    except ValueError:
        print("❌ Invalid input!")
